In [ ]:
from pandas import*
import pandas as pd
import datetime
train_loc = 'C:/Users/Profein/Desktop/Credit score/traindemographics.csv'
def age(loc) :
    t_d = pd.read_csv(loc)
    t_d.describe()
    t_d[t_d["bank_branch_clients"].notnull()]
    now = 2017
    t_d['birthdate'] = pd.to_datetime(t_d['birthdate']).apply(lambda x: x.year)
    t_d['age'] = now - t_d['birthdate']
    return t_d
data = age('C:/Users/Profein/Desktop/Credit score/traindemographics.csv')
demo = age('C:/Users/Profein/Desktop/Credit score/testdemographics.csv')
data = pd.concat([data, demo]).drop_duplicates().reset_index(drop=True)
perf = pd.read_csv('C:/Users/Profein/Desktop/Credit score/trainperf.csv')
perf1 = pd.read_csv('C:/Users/Profein/Desktop/Credit score/testperf.csv')
prev = pd.read_csv('C:/Users/Profein/Desktop/Credit score/trainprevloans.csv')
prev1 = pd.read_csv('C:/Users/Profein/Desktop/Credit score/testprevloans.csv')
prev = pd.concat([prev, prev1]).drop_duplicates().reset_index(drop = True)
df = pd.merge(data, perf, on = 'customerid',  how='inner')
df1 = pd.merge(data, perf1, on = 'customerid',  how='inner')
prev['closed'] = pd.to_datetime(prev['closeddate']) - pd.to_datetime(prev['approveddate'])
prev['closed'] = prev.loc[:,'closed'].astype('timedelta64[D]').astype(int)
prev['flag'] = 0
prev.loc[prev['closed']<=prev['termdays'],'flag']=1
prev['flag']
sumprev = prev[['customerid', 'flag']].groupby(['customerid'], as_index = False).mean()
meanprev = prev[['customerid', 'flag']].groupby(['customerid'], as_index = False).sum()
sumprev1 = prev.loc[:,('customerid','loanamount','termdays', 'flag')]
prevv = pd.merge(sumprev, meanprev, on = 'customerid', how = 'inner' )
sumprev1.loc[sumprev1['loanamount']<=20000,'value1'] = sumprev1['flag']
sumprev1.loc[sumprev1['loanamount']>20000,'value2'] = sumprev1['flag']
sumprev1.loc[sumprev1['termdays']<=30,'value3'] = sumprev1['flag']
sumprev1.loc[sumprev1['termdays']>30,'value4'] = sumprev1['flag']

new = sumprev1[['customerid', 'value1','value2','value3','value4']].groupby(['customerid'], as_index = False).mean()
previous = pd.merge(prevv, new,on = 'customerid', how = 'inner' )
previous = previous.fillna(2)
df = pd.merge(df, previous, on = 'customerid', how = 'left')
df1 = pd.merge(df1, previous, on = 'customerid', how = 'left')
df[['flag_x', 'flag_y','value1','value2','value3','value4']] = df[['flag_x','flag_y', 'value1','value2','value3','value4']].fillna(10)
df['prediction'] = 0
df.loc[df['good_bad_flag']=='Good','prediction']=1
df.head(3) 
df = df.drop('good_bad_flag',axis=1)
df['level_of_education_clients']= df['level_of_education_clients'].fillna('Not_given')
df[['level_of_education_clients', 'prediction']].groupby(['level_of_education_clients'], as_index = False).mean()
df['is_referred']=1
df.loc[df['referredby'].isnull(),'is_referred']=0
df['employment_status_clients'] = df['employment_status_clients'].fillna('Notgiven')
df['catergorical_age']= pd.cut(df['age'],5)
df['longs']= pd.cut(df['longitude_gps'],10)
df['lats']= pd.cut(df['latitude_gps'],10)
df['categorical_amount']= pd.cut(df['loanamount'],6)
df['interest'] = df['totaldue'] /df['loanamount']
df['cat_int'] = pd.cut(df['interest'],3)
df[['lats', 'prediction']].groupby(['lats'], as_index = False).mean()
df['loan_num'] = pd.cut(df['loannumber'],5)
df[['loannumber', 'prediction']].groupby(['loannumber'], as_index = False).mean() 

df1[['flag_x', 'flag_y','value1','value2','value3','value4']] = df1[['flag_x','flag_y', 'value1','value2','value3','value4']].fillna(10)
df1['level_of_education_clients']= df1['level_of_education_clients'].fillna('Not_given')
df1['employment_status_clients'] = df1['employment_status_clients'].fillna('Notgiven')
df1['catergorical_age']= pd.cut(df['age'],5)
df1['longs']= pd.cut(df['longitude_gps'],10)
df1['lats']= pd.cut(df['latitude_gps'],10)
df1['categorical_amount']= pd.cut(df['loanamount'],6)
df1['interest'] = df['totaldue'] /df['loanamount']
df1['cat_int'] = pd.cut(df['interest'],3)
df1['loan_num'] = pd.cut(df['loannumber'],5)

f = df1[[ 'flag_x','flag_y','value1','value2','value3','value4','age','termdays','interest', 'loanamount', 'bank_name_clients', 'longitude_gps', 'latitude_gps','loannumber' ]]         
f[['loanamount','longitude_gps','latitude_gps']] = f[['loanamount','longitude_gps','latitude_gps']].astype(int)

f1 = df[['flag_x','flag_y','value1','value2','value3','value4', 'termdays','cat_int', 'categorical_amount', 'bank_name_clients', 'catergorical_age', 'longs', 'lats', 'employment_status_clients', 'loan_num' ]]
f2 = df[[ 'flag_x','flag_y','value1','value2','value3','value4','age','termdays','interest', 'loanamount', 'bank_name_clients', 'longitude_gps', 'latitude_gps','loannumber' ]]         
f2[['loanamount','longitude_gps','latitude_gps']] = f2[['loanamount','longitude_gps','latitude_gps']].astype(int)
f1 = pd.get_dummies(f1)
f2 = pd.get_dummies(f2)
f = pd.get_dummies(f)

Y = df['prediction']
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split 
Xtrain, Xvalidation, Ytrain, Yvalidation = train_test_split(f2,Y,test_size=0.3, random_state=True)
model = RandomForestClassifier(n_estimators=30000,
                               min_samples_leaf = 1,
                               max_leaf_nodes=150,
                               max_depth=None,
                               random_state=50,)
model.fit(Xtrain, Ytrain)
tree = DecisionTreeClassifier(max_depth =3 ,random_state = 0)
tree.fit(f2, Y)
from sklearn.metrics import accuracy_score
Yprediction = model.predict(Xvalidation)
Yprediction_t = tree.predict(f)
print(accuracy_score(Yvalidation, Yprediction))
print(accuracy_score(Yvalidation,Yprediction_t))
from sklearn import svm
clf = svm.SVC(C= 1, gamma = 0.8, kernel = 'rbf')
clf.fit(f2, Y)
Yprediction1 = clf.predict(f)
print(accuracy_score(Yvalidation, Yprediction1))
df1['Good_bad_flag'] = Yprediction
df1[['customerid','Good_bad_flag']].to_csv('C:/Users/Profein/Desktop/Credit score/submission.csv')
Xtrain['label']= Ytrain
Xvalidation['label'] = Yvalidation
Xtrain.to_csv('C:/Users/Profein/Desktop/Credit score/train1.csv')
Xvalidation.to_csv('C:/Users/Profein/Desktop/Credit score/test1.csv')